In [ ]:
!nvidia-smi


Wed Dec 11 07:47:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import os
new_path = '/content/drive/MyDrive/linear_rep_geometry-main'
os.chdir(new_path)
print("Changed Working Directory to:", os.getcwd())

Changed Working Directory to: /content/drive/MyDrive/linear_rep_geometry-main


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1. Counterfactual Pairs 처리
def get_counterfactual_pairs(filename, tokenizer, max_pairs=10):
    with open(filename, 'r') as f:
        lines = f.readlines()

    word_pairs = [line.strip().split('\t') for line in lines if line.strip()][:max_pairs]
    base_words = [pair[0] for pair in word_pairs]
    target_words = [pair[1] for pair in word_pairs]

    # Tokenize and get token indices
    base_tokens = [tokenizer.encode(word, add_special_tokens=False) for word in base_words]
    target_tokens = [tokenizer.encode(word, add_special_tokens=False) for word in target_words]

    return base_tokens, target_tokens

# 2. Embedding 계산 및 Conceptual Vector 생성
def compute_conceptual_vectors(base_tokens, target_tokens, model, tokenizer, device="cuda"):
    base_sentences = [tokenizer.decode(tokens) for tokens in base_tokens]
    target_sentences = [tokenizer.decode(tokens) for tokens in target_tokens]

    sentences = base_sentences + target_sentences
    embeddings = get_embeddings(sentences, model, tokenizer, device)

    base_embeddings = embeddings[:len(base_tokens)]
    target_embeddings = embeddings[len(base_tokens):]

    diff_embeddings = target_embeddings - base_embeddings
    mean_vector = torch.mean(diff_embeddings, dim=0)
    return mean_vector

def get_embeddings(sentences, model, tokenizer, device="cuda", max_length=512):
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # max_length와 truncation 추가
    inputs = tokenizer(
        sentences,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]

    attention_mask = inputs["attention_mask"]
    expanded_mask = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
    summed_embeddings = torch.sum(hidden_states * expanded_mask, dim=1)
    summed_mask = torch.sum(expanded_mask, dim=1)
    embeddings = summed_embeddings / summed_mask

    return embeddings



def compute_projection_matrix(conceptual_vectors, hidden_dim):
    """
    Conceptual Vectors로부터 Projection Matrix를 계산 (QR 기반).

    Args:
        conceptual_vectors: torch.Tensor, Conceptual Vectors (shape: num_concepts x hidden_dim).
        hidden_dim: int, 모델의 hidden dimension (예: 1024).

    Returns:
        projection_matrix: torch.Tensor, Projection Matrix (shape: hidden_dim x hidden_dim).
    """
    # QR 분해를 사용하여 orthogonal basis 계산
    orthogonal_basis = torch.linalg.qr(conceptual_vectors.T)[0]

    # Projection Matrix 계산 (hidden_dim 크기로 확장)
    projection_matrix = orthogonal_basis @ orthogonal_basis.T
    return projection_matrix



# 5. Orthogonal Procrustes Analysis (OPA)
def orthogonal_procrustes(A, B):
    """
    두 Projection Matrix를 align하기 위한 Orthogonal Procrustes Analysis.
    """
    M = B.T @ A
    U, _, V = torch.linalg.svd(M)
    R = U @ V.T
    return R

def run_experiment_for_all_files(directory, model_1, tokenizer_1, model_2, tokenizer_2):
    concept_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.txt')]

    all_conceptual_vectors_1 = []
    all_conceptual_vectors_2 = []

    for concept_file in concept_files:
        print(f"\nProcessing file: {concept_file}")

        # Counterfactual Pairs 로드
        base_tokens_1, target_tokens_1 = get_counterfactual_pairs(concept_file, tokenizer_1)
        base_tokens_2, target_tokens_2 = get_counterfactual_pairs(concept_file, tokenizer_2)

        # Conceptual Vectors 계산
        conceptual_vectors_1 = compute_conceptual_vectors(base_tokens_1, target_tokens_1, model_1, tokenizer_1)
        conceptual_vectors_2 = compute_conceptual_vectors(base_tokens_2, target_tokens_2, model_2, tokenizer_2)

        all_conceptual_vectors_1.append(conceptual_vectors_1)
        all_conceptual_vectors_2.append(conceptual_vectors_2)

    # Projection Matrix 계산
    subspace_1 = torch.stack(all_conceptual_vectors_1)
    subspace_2 = torch.stack(all_conceptual_vectors_2)

    hidden_dim = subspace_1.shape[1]  # Hidden dimension (e.g., 1024)
    P1 = compute_projection_matrix(subspace_1, hidden_dim)
    P2 = compute_projection_matrix(subspace_2, hidden_dim)

    # Orthogonal Procrustes Analysis로 R 계산
    R = orthogonal_procrustes(P1, P2)

    # 학습된 R을 model_2에 반영
    update_model_with_R(model_2, R)

    print("\nExperiment completed. Model 2 embedding layer updated.")


In [ ]:
def update_model_with_R(model, R):
    """
    모델의 embedding layer를 정렬 행렬 R을 사용해 업데이트.

    Args:
        model: Pretrained 모델.
        R: torch.Tensor, 학습된 정렬 행렬 (shape: hidden_dim x hidden_dim).
    """
    # 모델의 embedding layer 가져오기
    embedding_layer = model.get_input_embeddings()

    # 기존 embedding layer의 가중치
    embedding_weights = embedding_layer.weight.data

    # 정렬 행렬 R을 이용하여 embedding 가중치 변환
    transformed_weights = embedding_weights @ R.T

    # 업데이트된 embedding 가중치를 embedding layer에 반영
    embedding_layer.weight.data = transformed_weights

    print("Model embedding layer updated with R.")


In [ ]:
# 모델 및 토크나이저 설정
model_1_name = "gpt2-medium"
model_2_name = "bigscience/bloom-560m"

tokenizer_1 = AutoTokenizer.from_pretrained(model_1_name)
model_1 = AutoModelForCausalLM.from_pretrained(model_1_name).to("cuda")

tokenizer_2 = AutoTokenizer.from_pretrained(model_2_name)
model_2 = AutoModelForCausalLM.from_pretrained(model_2_name).to("cuda")

# word_pairs 디렉토리에서 모든 파일 실행
directory = "word_pairs"
run_experiment_for_all_files(directory, model_1, tokenizer_1, model_2, tokenizer_2)



Processing file: word_pairs/[adj - comparative].txt

Processing file: word_pairs/[Ving - Ved].txt

Processing file: word_pairs/[French - German].txt

Processing file: word_pairs/[3pSg - Ved].txt

Processing file: word_pairs/[adj - superlative].txt

Processing file: word_pairs/[adj - adj + ly].txt

Processing file: word_pairs/[Ving - 3pSg].txt

Processing file: word_pairs/[English - French].txt

Processing file: word_pairs/[French - Spanish].txt

Processing file: word_pairs/[German - Spanish].txt

Processing file: word_pairs/[frequent - infrequent].txt

Processing file: word_pairs/[verb - V + ment].txt

Processing file: word_pairs/[adj - un + adj].txt

Processing file: word_pairs/[small - big].txt

Processing file: word_pairs/[thing - part].txt

Processing file: word_pairs/[verb - V + er].txt

Processing file: word_pairs/[verb - 3pSg].txt

Processing file: word_pairs/[verb - V + able].txt

Processing file: word_pairs/[verb - Ving].txt

Processing file: word_pairs/[verb - V + tion].txt


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr  # 수정된 부분
import numpy as np

# 데이터셋 준비 (임의의 문장 쌍)


# 레이블: 유사도 점수 (0: 매우 다름, 1: 매우 유사)



# 문장 쌍을 모델로부터 임베딩 벡터로 변환하는 함수
def compute_sentence_embeddings(sentences, model, tokenizer, device="cuda"):
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer(
        sentences,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]

    attention_mask = inputs["attention_mask"]
    expanded_mask = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
    summed_embeddings = torch.sum(hidden_states * expanded_mask, dim=1)
    summed_mask = torch.sum(expanded_mask, dim=1)
    embeddings = summed_embeddings / summed_mask

    return embeddings.cpu()

# 코사인 유사도 계산 함수
def cosine_similarity(emb1, emb2):
    dot_product = np.dot(emb1, emb2)
    norm1 = np.linalg.norm(emb1)
    norm2 = np.linalg.norm(emb2)
    return dot_product / (norm1 * norm2)

# 모델 성능 평가 함수
def evaluate_model(sentence_pairs, labels, model, tokenizer, device="cuda"):
    model = model.to(device)
    predicted_similarities = []

    for sent1, sent2 in sentence_pairs:
        embeddings = compute_sentence_embeddings([sent1, sent2], model, tokenizer, device)
        similarity = cosine_similarity(embeddings[0].numpy(), embeddings[1].numpy())
        predicted_similarities.append(similarity)

    # Pearson Correlation
    pearson_corr = pearsonr(labels, predicted_similarities)[0]

    # Mean Squared Error
    mse = mean_squared_error(labels, predicted_similarities)

    return pearson_corr, mse

# 모델 비교
print("Evaluating Original Model 2:")
original_pearson, original_mse = evaluate_model(sentence_pairs, labels, original_model_2, tokenizer_2)

print("Evaluating Updated Model 2:")
updated_pearson, updated_mse = evaluate_model(sentence_pairs, labels, model_2, tokenizer_2)

# 결과 출력
print("\nPerformance Comparison:")
print(f"Original Model 2 - Pearson Correlation: {original_pearson:.4f}, MSE: {original_mse:.4f}")
print(f"Updated Model 2 - Pearson Correlation: {updated_pearson:.4f}, MSE: {updated_mse:.4f}")


Evaluating Original Model 2:


ValueError: x and y must have the same length.

In [44]:
original_model_2_name = "bigscience/bloom-560m"


original_tokenizer_2 = AutoTokenizer.from_pretrained(original_model_2_name)
original_model_2 = AutoModelForCausalLM.from_pretrained(original_model_2_name).to("cuda")

In [45]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

# **1. Load STS Benchmark Dataset (TSV Format)**
def load_sts_data(filepath):
    """
    Load STS Benchmark dataset from a TSV file.

    """

    data = pd.read_csv(filepath, sep='\t', quoting=3, skiprows=1, header=None)
    data.columns = ['index', 'genre', 'filename', 'year','old_index','source1','source2', 'sentence1', 'sentence2','score']

    data['score'] = data['score'].astype(float)

    return data[['sentence1', 'sentence2', 'score']]


# **2. Compute Sentence Embeddings**
def compute_sentence_embeddings(sentences, model, tokenizer, device="cuda"):
    """
    Compute embeddings for a list of sentences using the given model and tokenizer.
    """
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]  # Use the last hidden layer
        attention_mask = inputs["attention_mask"]
        expanded_mask = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
        summed_embeddings = torch.sum(hidden_states * expanded_mask, dim=1)
        summed_mask = torch.sum(expanded_mask, dim=1)
        embeddings = summed_embeddings / summed_mask
    return embeddings

# **3. Compute Semantic Similarity**
def compute_similarity(sentence_pairs, model, tokenizer, device="cuda"):
    """
    Compute semantic similarity scores for a list of sentence pairs.
    """
    similarities = []
    for sentence1, sentence2 in sentence_pairs:
        embeddings = compute_sentence_embeddings([sentence1, sentence2], model, tokenizer, device)
        sim = torch.cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1].unsqueeze(0)).item()
        similarities.append(sim)
    return similarities

# **4. Evaluate Model**
def evaluate_model(data, model, tokenizer, device="cuda"):
    """
    Evaluate the model on STS Benchmark dataset and return Pearson correlation and MSE.
    """
    sentence_pairs = list(zip(data['sentence1'], data['sentence2']))
    true_scores = data['score'].tolist()

    # Normalize true scores to [0, 1] for consistency
    true_scores = [score / 5.0 for score in true_scores]

    # Compute model's similarity predictions
    predicted_scores = compute_similarity(sentence_pairs, model, tokenizer, device)

    # Compute evaluation metrics
    pearson_corr = pearsonr(predicted_scores, true_scores)[0]
    mse = mean_squared_error(true_scores, predicted_scores)
    return pearson_corr, mse

# **5. Main Experiment**
def run_sts_experiment(data_filepath, original_model, updated_model, tokenizer, device="cuda"):
    """
    Run the STS Benchmark experiment comparing the original and updated models.
    """
    # Load STS data
    data = load_sts_data(data_filepath)

    print("Evaluating Original Model...")
    original_corr, original_mse = evaluate_model(data, original_model, tokenizer, device)
    print(f"Original Model - Pearson Correlation: {original_corr:.4f}, MSE: {original_mse:.4f}")

    print("\nEvaluating Updated Model...")
    updated_corr, updated_mse = evaluate_model(data, updated_model, tokenizer, device)
    print(f"Updated Model - Pearson Correlation: {updated_corr:.4f}, MSE: {updated_mse:.4f}")

    return {
        "original": {"pearson": original_corr, "mse": original_mse},
        "updated": {"pearson": updated_corr, "mse": updated_mse},
    }

# **6. Example Usage**
if __name__ == "__main__":
    # Paths and Model Loading
    sts_filepath = "train.tsv"  # Update this to the actual path
    original_model = original_model_2
    updated_model = model_2  # Replace with your updated model after applying R

    # Run the experiment
    results = run_sts_experiment(sts_filepath, original_model, updated_model, tokenizer_2)
    print("Results:", results)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Evaluating Original Model...
Original Model - Pearson Correlation: 0.0483, MSE: 0.2939

Evaluating Updated Model...
Updated Model - Pearson Correlation: 0.1261, MSE: 0.2759
Results: {'original': {'pearson': 0.048264908377855, 'mse': 0.29387649791896014}, 'updated': {'pearson': 0.12607134881281734, 'mse': 0.27594532095588475}}


In [41]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1. Counterfactual Pairs 처리
def get_counterfactual_pairs(filename, tokenizer, max_pairs=10):
    with open(filename, 'r') as f:
        lines = f.readlines()

    word_pairs = [line.strip().split('\t') for line in lines if line.strip()][:max_pairs]
    base_words = [pair[0] for pair in word_pairs]
    target_words = [pair[1] for pair in word_pairs]

    # Tokenize and get token indices
    base_tokens = [tokenizer.encode(word, add_special_tokens=False) for word in base_words]
    target_tokens = [tokenizer.encode(word, add_special_tokens=False) for word in target_words]

    return base_tokens, target_tokens

# 2. Embedding 계산 및 Conceptual Vector 생성
def compute_conceptual_vectors(base_tokens, target_tokens, model, tokenizer, device="cuda"):
    base_sentences = [tokenizer.decode(tokens) for tokens in base_tokens]
    target_sentences = [tokenizer.decode(tokens) for tokens in target_tokens]

    sentences = base_sentences + target_sentences
    embeddings = get_embeddings(sentences, model, tokenizer, device)

    base_embeddings = embeddings[:len(base_tokens)]
    target_embeddings = embeddings[len(base_tokens):]

    diff_embeddings = target_embeddings - base_embeddings
    mean_vector = torch.mean(diff_embeddings, dim=0)
    return mean_vector

def get_embeddings(sentences, model, tokenizer, device="cuda", max_length=512):
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # max_length와 truncation 추가
    inputs = tokenizer(
        sentences,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]

    attention_mask = inputs["attention_mask"]
    expanded_mask = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
    summed_embeddings = torch.sum(hidden_states * expanded_mask, dim=1)
    summed_mask = torch.sum(expanded_mask, dim=1)
    embeddings = summed_embeddings / summed_mask

    return embeddings



def compute_projection_matrix(conceptual_vectors, hidden_dim):
    """
    Conceptual Vectors로부터 Projection Matrix를 계산 (QR 기반).

    Args:
        conceptual_vectors: torch.Tensor, Conceptual Vectors (shape: num_concepts x hidden_dim).
        hidden_dim: int, 모델의 hidden dimension (예: 1024).

    Returns:
        projection_matrix: torch.Tensor, Projection Matrix (shape: hidden_dim x hidden_dim).
    """
    # QR 분해를 사용하여 orthogonal basis 계산
    orthogonal_basis = torch.linalg.qr(conceptual_vectors.T)[0]

    # Projection Matrix 계산 (hidden_dim 크기로 확장)
    projection_matrix = orthogonal_basis @ orthogonal_basis.T
    return projection_matrix



# 5. Orthogonal Procrustes Analysis (OPA)
def orthogonal_procrustes(A, B):
    """
    두 Projection Matrix를 align하기 위한 Orthogonal Procrustes Analysis.
    """
    M = B.T @ A
    U, _, V = torch.linalg.svd(M)
    R = U @ V.T
    return R

import os
import torch

import os
import torch

def run_experiment_for_all_files(directory, model_1, tokenizer_1, model_2, tokenizer_2, max_iters=100, lr=1e-5):
    """
    Run the experiment for all word pair files in the given directory.

    Args:
        directory (str): Directory containing word pair files.
        model_1 (transformers.PreTrainedModel): First model.
        tokenizer_1 (transformers.PreTrainedTokenizer): Tokenizer for the first model.
        model_2 (transformers.PreTrainedModel): Second model.
        tokenizer_2 (transformers.PreTrainedTokenizer): Tokenizer for the second model.
        max_iters (int): Maximum number of iterations for gradient descent.
        lr (float): Learning rate for gradient descent.

    Returns:
        None
    """
    all_conceptual_vectors_1 = []
    all_conceptual_vectors_2 = []

    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            print(f"Processing file: {filepath}")

            # Step 1: Load counterfactual pairs and compute tokens
            base_tokens_1, target_tokens_1 = get_counterfactual_pairs(filepath, tokenizer_1)
            base_tokens_2, target_tokens_2 = get_counterfactual_pairs(filepath, tokenizer_2)

            # Step 2: Compute conceptual vectors for both models
            conceptual_vector_1 = compute_conceptual_vectors(base_tokens_1, target_tokens_1, model_1, tokenizer_1)
            conceptual_vector_2 = compute_conceptual_vectors(base_tokens_2, target_tokens_2, model_2, tokenizer_2)

            all_conceptual_vectors_1.append(conceptual_vector_1)
            all_conceptual_vectors_2.append(conceptual_vector_2)

    # Step 3: Compute projection matrices for both models
    conceptual_vectors_1 = torch.stack(all_conceptual_vectors_1)
    conceptual_vectors_2 = torch.stack(all_conceptual_vectors_2)

    hidden_dim = conceptual_vectors_1.size(1)  # Assuming conceptual vectors are of shape (num_concepts, hidden_dim)
    P1 = compute_projection_matrix(conceptual_vectors_1, hidden_dim)
    P2 = compute_projection_matrix(conceptual_vectors_2, hidden_dim)

    print("Projection matrices computed.")

    # Step 4: Optimize R using SO(n) gradient descent
    R = optimize_R_on_SO(P1, P2, max_iters=max_iters, lr=lr)

    # Step 5: Update model_2's embedding layer with R
    update_model_with_R(model_2, R)

    print("Model 2 updated with optimized R.")

# Supporting functions

def compute_gradient_so(P1, P2, R):
    """
    Compute the gradient of the loss function J(R) and project it onto so(n).

    Args:
        P1 (torch.Tensor): Projection matrix of model 1 (shape: d x d).
        P2 (torch.Tensor): Projection matrix of model 2 (shape: d x d).
        R (torch.Tensor): Current rotation matrix (shape: d x d).

    Returns:
        torch.Tensor: Gradient projected onto so(n) (shape: d x d).
    """

    max_norm=1.0
    # Compute the loss gradient before projection
    diff = P1 - R @ P2 @ R.T
    grad = -4 * (diff @ P2 @ R)

    grad_norm = torch.norm(grad)
    if grad_norm > max_norm:
        grad = grad * (max_norm / grad_norm)


    # Project the gradient onto so(n)
    grad_antisym = 0.5 * (grad - grad.T)
    return grad_antisym

def optimize_R_on_SO(P1, P2, max_iters=100, lr=1e-2):
    """
    Perform optimization on SO(n) to find the optimal R.

    Args:
        P1 (torch.Tensor): Projection matrix of model 1 (shape: d x d).
        P2 (torch.Tensor): Projection matrix of model 2 (shape: d x d).
        max_iters (int): Maximum number of iterations.
        lr (float): Learning rate.

    Returns:
        torch.Tensor: Optimized rotation matrix R (shape: d x d).
    """
    # Initialize R as the identity matrix
    #d = P1.shape[0]
    #R = torch.eye(d, device=P1.device)

    #U, _, V = torch.svd(P1 @ P2.T)
    #R = (U @ V.T).clone()  # 초기화된 R
    device="cuda"
    hidden_dim=1024
    R = torch.eye(hidden_dim).to(device) + 0.01 * torch.randn(hidden_dim, hidden_dim).to(device)
    R = torch.linalg.qr(R)[0]


    for i in range(max_iters):
        # Compute the gradient projected onto so(n)
        grad_so = compute_gradient_so(P1, P2, R)

        # Gradient descent step
        R = R - lr * grad_so

        # Re-project R onto SO(n) using SVD
        U, _, V = torch.svd(R)
        R = U @ V.T

        # Compute the loss
        diff = P1 - R @ P2 @ R.T
        loss = torch.trace(diff @ diff.T)

        # Print progress
        print(f"Iteration {i+1}/{max_iters}, Loss: {loss.item()}")

    return R

def compute_projection_matrix(conceptual_vectors, hidden_dim):
    """
    Compute the projection matrix for a set of conceptual vectors.

    Args:
        conceptual_vectors (torch.Tensor): Conceptual vectors (shape: num_concepts x hidden_dim).
        hidden_dim (int): Dimensionality of the hidden space.

    Returns:
        torch.Tensor: Projection matrix (shape: hidden_dim x hidden_dim).
    """
    orthogonal_basis = torch.linalg.qr(conceptual_vectors.T)[0]
    projection_matrix = orthogonal_basis @ orthogonal_basis.T
    return projection_matrix


In [42]:
# 모델 및 토크나이저 설정
model_1_name = "gpt2-medium"
model_2_name = "bigscience/bloom-560m"

tokenizer_1 = AutoTokenizer.from_pretrained(model_1_name)
model_1 = AutoModelForCausalLM.from_pretrained(model_1_name).to("cuda")

tokenizer_2 = AutoTokenizer.from_pretrained(model_2_name)
model_2 = AutoModelForCausalLM.from_pretrained(model_2_name).to("cuda")

# word_pairs 디렉토리에서 모든 파일 실행
directory = "word_pairs"
run_experiment_for_all_files(directory, model_1, tokenizer_1, model_2, tokenizer_2)

Processing file: word_pairs/[adj - comparative].txt
Processing file: word_pairs/[Ving - Ved].txt
Processing file: word_pairs/[French - German].txt
Processing file: word_pairs/[3pSg - Ved].txt
Processing file: word_pairs/[adj - superlative].txt
Processing file: word_pairs/[adj - adj + ly].txt
Processing file: word_pairs/[Ving - 3pSg].txt
Processing file: word_pairs/[English - French].txt
Processing file: word_pairs/[French - Spanish].txt
Processing file: word_pairs/[German - Spanish].txt
Processing file: word_pairs/[frequent - infrequent].txt
Processing file: word_pairs/[verb - V + ment].txt
Processing file: word_pairs/[adj - un + adj].txt
Processing file: word_pairs/[small - big].txt
Processing file: word_pairs/[thing - part].txt
Processing file: word_pairs/[verb - V + er].txt
Processing file: word_pairs/[verb - 3pSg].txt
Processing file: word_pairs/[verb - V + able].txt
Processing file: word_pairs/[verb - Ving].txt
Processing file: word_pairs/[verb - V + tion].txt
Processing file: wor

In [ ]:
def update_model_with_R(model, R):
    """
    모델의 embedding layer를 정렬 행렬 R을 사용해 업데이트.

    Args:
        model: Pretrained 모델.
        R: torch.Tensor, 학습된 정렬 행렬 (shape: hidden_dim x hidden_dim).
    """
    # 모델의 embedding layer 가져오기
    embedding_layer = model.get_input_embeddings()

    # 기존 embedding layer의 가중치
    embedding_weights = embedding_layer.weight.data

    # 정렬 행렬 R을 이용하여 embedding 가중치 변환
    transformed_weights = embedding_weights @ R.T

    # 업데이트된 embedding 가중치를 embedding layer에 반영
    embedding_layer.weight.data = transformed_weights

    print("Model embedding layer updated with R.")
